# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo ```SINASC_RO_2019.csv``` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo ```estrutura_sinasc_para_CD.pdf``` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
``` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
```
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis ```['ESTCIVMAE', 'CONSULTAS']``` possuem o código ```9```, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código ```9```.<br>
6. Substitua os valores faltantes da quantitativa (```QTDFILVIVO```) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção ```inplace = True```.

In [76]:
import pandas as pd
import requests

In [77]:
#1) 

# Carregar a base
sinasc = pd.read_csv('SINASC_RO_2019.csv')

# Numero de registros
print(sinasc.shape)

# Deletar registros duplicados
sinasc.drop_duplicates()

# Numero de registros duplicados
print('O número de registros duplicados é:')
sinasc.duplicated().sum()

(27028, 69)
O número de registros duplicados é:


0

In [78]:
# 2)

# Numero de valores missing por variavel
sinasc.isna().sum()

ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64

In [79]:
# 3)

# Nova seleção com base nas colunas de interesse
sinasc_estudo = sinasc.loc[:,['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']]
sinasc_resum

,loc_nasc,idade_mae,est_civ_mae,esc_mae,qtd_fil_vivo,gestacao,gravidez,consultas,apgar5,apgar5_class
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10,normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9,normal
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10,normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10,normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10,normal
...,...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9,normal
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9,normal
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10,normal
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9,normal


In [80]:
#3)

# Contagem de valores missing para nova seleção
sinasc_estudo.isna().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64

In [81]:
#4)

# Removar todos os registros não preenchidos da variavel APGAR5
sinasc_estudo.dropna(subset=['APGAR5'],inplace=True)

In [82]:
#4)

# Recontagem do numero de linhas
sinasc_estudo.shape

(26925, 9)

In [83]:
#4)

# Recontagem do numero de missing
sinasc_estudo.isna().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      315
ESCMAE         310
QTDFILVIVO    1566
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64

In [84]:
#5)

# Assumir que os valores missing para as variaveis 'ESTCIVMAE', 'CONSULTAS', sao iguais ao codigo 9 *ignorado.
sinasc_estudo['ESTCIVMAE'].fillna(9.0, inplace=True)
sinasc_estudo['CONSULTAS'].fillna(9, inplace=True)
sinasc_estudo.isna().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE         310
QTDFILVIVO    1566
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64

In [85]:
#6)

# Substituir os valores missing de QTDFILVIVO por zero
sinasc_estudo['QTDFILVIVO'].fillna(0, inplace=True)
sinasc_estudo.isna().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE         310
QTDFILVIVO       0
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64

In [86]:
#7)

# Substituir os valores missing de 'ESCMAE' por 9 *ignorado. A variavel corresponde a Escolaridade, anos de estudo
# concluídos da mae e valores preenchidos de forma arbitraria podem afetar a leitura dos dados.
# A porcentagem de registros *ignorados é de 1,1%. 

sinasc_estudo['ESCMAE'].fillna(9, inplace=True)

porcentagemNaN_escmae = 310/26925
porcentagemNaN_escmae

0.011513463324048283

In [87]:
#7)

# Substituir os valores missing de 'GESTACAO' por 9 *ignorado. A variavel corresponde a Semanas de gestação,
# e valores preenchidos de forma arbitraria podem afetar a leitura dos dados.
# A porcentagem de registros *ignorados é de 4,5%. 

sinasc_estudo['GESTACAO'].fillna(9, inplace=True)

porcentagemNaN_gestacao = 1216/26925
porcentagemNaN_gestacao

0.04516248839368617

In [88]:
#8) 

# Categorização da variável 'APGAR5' segundo a codificação abaixo:
# Entre 8 e 10 está em uma faixa 'normal'.
# Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'.
# Entre 4 e 5 significa 'asfixia moderada'.
# Entre 0 e 3 significa 'asfixia severa'.

sinasc_estudo['APGAR5'] = sinasc_estudo['APGAR5'].astype(int)

sinasc_estudo.loc[(sinasc_estudo['APGAR5'] >= 8.0) & (sinasc_estudo['APGAR5'] <= 10.0),
           'apgar5_class'] = 'normal'
sinasc_estudo.loc[(sinasc_estudo['APGAR5'] >= 6.0) & (sinasc_estudo['APGAR5'] <= 7.0),
           'apgar5_class'] = 'asfixia leve'
sinasc_estudo.loc[(sinasc_estudo['APGAR5'] >= 4.0) & (sinasc_estudo['APGAR5'] <= 5.0),
           'apgar5_class'] = 'asfixia moderada'
sinasc_estudo.loc[(sinasc_estudo['APGAR5'] >= 0.0) & (sinasc_estudo['APGAR5'] <= 3.0),
           'apgar5_class'] = 'asfixia severa'

sinasc_estudo

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5,apgar5_class
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10,normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9,normal
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10,normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10,normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10,normal
...,...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9,normal
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9,normal
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10,normal
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9,normal


In [89]:
#8)

# Calculo das frequencias das categorias da variavel "apgar5_class"

sinasc_estudo['apgar5_class'].value_counts()

normal              26463
asfixia leve          320
asfixia severa         74
asfixia moderada       68
Name: apgar5_class, dtype: int64

In [90]:
#9) 

# Renomear as variáveis para lower case e separacao com underscore 

sinasc_estudo.columns

Index(['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO',
       'GRAVIDEZ', 'CONSULTAS', 'APGAR5', 'apgar5_class'],
      dtype='object')

In [91]:
#9) 

columns = ['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO',
       'GRAVIDEZ', 'CONSULTAS', 'APGAR5']
for letters in range(len(columns)):
    columns[letters] = columns[letters].lower()
print(columns)

['locnasc', 'idademae', 'estcivmae', 'escmae', 'qtdfilvivo', 'gestacao', 'gravidez', 'consultas', 'apgar5']


In [92]:
#9)

sinasc_estudo.columns = ['loc_nasc', 'idade_mae', 'est_civ_mae', 'esc_mae', 'qtd_fil_vivo', 'gestacao', 'gravidez', 'consultas', 'apgar5', 'apgar5_class']
sinasc_estudo

,loc_nasc,idade_mae,est_civ_mae,esc_mae,qtd_fil_vivo,gestacao,gravidez,consultas,apgar5,apgar5_class
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10,normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9,normal
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10,normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10,normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10,normal
...,...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9,normal
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9,normal
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10,normal
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9,normal
